# NYC Health COVID-19 Data Scraper
### March 30, 2020
### Matthew J. Beattie
### University of Oklahoma

Cases obtained from _https://www1.nyc.gov/assets/doh/downloads/pdf/imm/covid-19-daily-data-summary.pdf_

Deaths obtained from _https://www1.nyc.gov/assets/doh/downloads/pdf/imm/covid-19-daily-data-summary-deaths.pdf_

In [1]:
import pandas as pd
import re
import PyPDF2 as pdf
import tempfile
import urllib.request
from datetime import date

yr = 2020
mo = 5
day = 5

## Define Common Routines

In [2]:
"""
textfrompdf()
Reads in PDF file from NYC website or local file.  To read a local file, the user
must flag the readfrom variable as 'local'
"""
def textfrompdf(url,readfrom='online'):
    if readfrom != 'local':
        dataIn = urllib.request.urlopen(url).read()
    else:
        localf = open(url, 'rb')
        dataIn = localf.read()
        localf.close()

    fp = tempfile.TemporaryFile()

    # Write the pdf data to a temp file
    fp.write(dataIn)

    pdfReader = pdf.PdfFileReader(fp)
    pdfReader.getNumPages()

    # Get the first page
    return pdfReader.getPage(0).extractText()

## Read in NYC Cases Data

In [3]:
# Read in text from the PDF file and display
url = 'https://www1.nyc.gov/assets/doh/downloads/pdf/imm/covid-19-daily-data-summary-05062020-1.pdf'
localurl = 'covid-19-daily-data-summary.pdf'
casespage = textfrompdf(url,readfrom='online')

casespage

'Coronavirus Disease 2019 (COVID\n-19) \n  Daily Data Summary\n  The data in this report reflect events and activities\n as of\n May\n 5, 2020\n at 6:00\n PM.  All data in this report are preliminary and subject to change as cases continue to be investigated. \n These\n data \ninclude \ncases in NYC residents and foreign residents treated in NYC facilities\n.  NYC COVID\n-19 Cases\n . Total Cases\n Total\n 173288 Median Age \n(Range)\n 51 (0\n-110) Age Group\n  -  0 to 17\n 4188 (2%)\n -  18 to 44\n 63726 (37%)\n -  45 to 64\n 63670 (37%)\n -  65 to 74\n 21590 (12%)\n -  75 and over\n 19779 (11%)\n -  Unknown\n 335 Age 50 and over\n  -  Yes\n 90189 (52%)\n -  No 82764 (48%)\n Sex\n  -  Female\n 83891 (48%)\n -  Male\n 89195 (52%)\n -  Unknown\n 202 Borough\n  -  Bronx\n 39587 (23%)\n -  Brooklyn\n 46139 (27%)\n -  Manhattan\n 21470 (12%)\n -  Queens\n 53692 (31%)\n -  Staten Island\n 12311 (7%)\n -  Unknown\n 89 Deaths\n  -  Confirmed\n 13938 (72%)\n -  Probable\n 5359 (28%)\n    '

## Extract the COVID cases by Age

In [4]:
# Extract the values for the cases by Age.  Ignore Unknown Age. 
tblText = re.sub("Coronavirus.*Cases.*0 to 17", "0 to 17", casespage, flags=re.DOTALL)
tblText = re.sub("Deaths.*","", tblText, flags=re.DOTALL)
tblText = re.sub("Unknown.*", "", tblText, flags=re.DOTALL)
tblText = re.sub("\(\d+%", "", tblText, flags=re.DOTALL)
tblText = re.sub(" \).+?(?=[a-zA-Z0-9])", ";", tblText, flags=re.DOTALL)
tblText = re.sub("\n.+?(?=[0-9])", ",", tblText, flags=re.DOTALL)
tblText = re.sub(" \).+", "", tblText, flags=re.DOTALL)
tblText = re.sub("0 to 17", "0-17", tblText, flags=re.DOTALL)
tblText = re.sub("18 to 44", "18-44", tblText, flags=re.DOTALL)
tblText = re.sub("45 to 64", "45-64", tblText, flags=re.DOTALL)
tblText = re.sub("65 to 74", "65-74", tblText, flags=re.DOTALL)
tblText = re.sub("75 and over", "Over_75", tblText, flags=re.DOTALL)


In [5]:
tblText

'0-17,4188;18-44,63726;45-64,63670;65-74,21590;Over_75,19779'

In [6]:
# Split the extracted text into rows and remove any rows that do not contain data
rows = re.split(";", tblText)
rows = [row for row in rows if re.search(r"[0-9]",row)!=None]

In [7]:
rows

['0-17,4188', '18-44,63726', '45-64,63670', '65-74,21590', 'Over_75,19779']

In [8]:
# Convert the rows of text into a list of lists
agedata = []
for row in rows:
    record = []
    field1 = re.search("^([^,])+", row).group()
    field2 = re.search("[^,]+$", row).group()
    record.append(field1)
    record.append(int(field2))
    agedata.append(record)

In [9]:
# Convert the list into a dataframe and display 
agedf = pd.DataFrame(agedata, columns=['category','count'])
agedf['feature']='age'
agedf['type']='cases'
agedf['date']=date(yr,mo,day)
agedf

,category,count,feature,type,date
0,0-17,4188,age,cases,2020-05-05
1,18-44,63726,age,cases,2020-05-05
2,45-64,63670,age,cases,2020-05-05
3,65-74,21590,age,cases,2020-05-05
4,Over_75,19779,age,cases,2020-05-05


## Extract the COVID cases by Sex

In [10]:
# Extract the values for the cases by Sex.  Ignore Unknown Sex. 
tblText = re.sub("Coronavirus.*Cases.*Female", "Female", casespage, flags=re.DOTALL)
tblText = re.sub("Deaths.*","", tblText, flags=re.DOTALL)
tblText = re.sub("Unknown.*", "", tblText, flags=re.DOTALL)
tblText = re.sub("\(\d+%", "", tblText, flags=re.DOTALL)
tblText = re.sub(" \).+?(?=[a-zA-Z0-9])", ";", tblText, flags=re.DOTALL)
tblText = re.sub("\n.+?(?=[0-9])", ",", tblText, flags=re.DOTALL)
tblText = re.sub(" \).+", "", tblText, flags=re.DOTALL)

In [11]:
# Split the extracted text into rows and remove any rows that do not contain data
rows = re.split(";", tblText)
rows = [row for row in rows if re.search(r"[0-9]",row)!=None]

In [12]:
# Convert the rows into a list of lists
sexdata = []
for row in rows:
    record = []
    field1 = re.search("^([^,])+", row).group()
    field2 = re.search("[^,]+$", row).group()
    record.append(field1)
    record.append(int(field2))
    sexdata.append(record)

# Convert the list into a dataframe and display
sexdf = pd.DataFrame(sexdata, columns=['category','count'])
sexdf['feature']='sex'
sexdf['type']='cases'
sexdf['date']=date(yr,mo,day)
sexdf

,category,count,feature,type,date
0,Female,83891,sex,cases,2020-05-05
1,Male,89195,sex,cases,2020-05-05


## Extract the COVID cases by Borough

In [13]:
# Extract the values for the cases by Borough.  Ignore Unknown Borough. 
tblText = re.sub("Coronavirus.*Cases.*Bronx", "Bronx", casespage, flags=re.DOTALL)
tblText = re.sub("Deaths.*","", tblText, flags=re.DOTALL)
tblText = re.sub("Unknown.*", "", tblText, flags=re.DOTALL)
tblText = re.sub("\(\d+%", "", tblText, flags=re.DOTALL)
tblText = re.sub(" \).+?(?=[a-zA-Z0-9])", ";", tblText, flags=re.DOTALL)
tblText = re.sub("\n.+?(?=[0-9])", ",", tblText, flags=re.DOTALL)
tblText = re.sub(" \).+", "", tblText, flags=re.DOTALL)
tblText = re.sub("Staten Island", "Staten_Island", tblText, flags=re.DOTALL)

# Split the extracted text into rows and remove any rows that do not contain data
rows = re.split(";", tblText)
rows = [row for row in rows if re.search(r"[0-9]",row)!=None]

In [14]:
# Covert the rows of text into a list of lists
boroughdata = []
for row in rows:
    record = []
    field1 = re.search("^([^,])+", row).group()
    field2 = re.search("[^,]+$", row).group()
    record.append(field1)
    record.append(int(field2))
    boroughdata.append(record)

# Convert the list into a dataframe and display    
boroughdf = pd.DataFrame(boroughdata, columns=['category','count'])
boroughdf['feature']='borough'
boroughdf['type']='cases'
boroughdf['date']=date(yr,mo,day)

# Concatenate Age, Sex, and Borough Stats into a cases dataframe and display
casesdf = pd.concat([agedf, sexdf, boroughdf]).reset_index().drop(columns=['index'])
casesdf

,category,count,feature,type,date
0,0-17,4188,age,cases,2020-05-05
1,18-44,63726,age,cases,2020-05-05
2,45-64,63670,age,cases,2020-05-05
3,65-74,21590,age,cases,2020-05-05
4,Over_75,19779,age,cases,2020-05-05
5,Female,83891,sex,cases,2020-05-05
6,Male,89195,sex,cases,2020-05-05
7,Bronx,39587,borough,cases,2020-05-05
8,Brooklyn,46139,borough,cases,2020-05-05
9,Manhattan,21470,borough,cases,2020-05-05


## Read in NYC Deaths Data
Note that the deaths program is a bit more efficient.  We handle the entire table in one pass instead of breaking it into Age, Sex, and Borough.

In [15]:
# Read in text from PDF and show in raw format
url = 'https://www1.nyc.gov/assets/doh/downloads/pdf/imm/covid-19-daily-data-summary-deaths-05062020-1.pdf'
localurl = 'covid-19-daily-data-summary-deaths-200330.pdf'
deathspage = textfrompdf(url,readfrom='online')
deathspage

'Coronavirus Disease 2019 (COVID\n-19) \n  Daily Data Summary\n  The data in this report reflect events and activities\n as of\n May \n5, 2020\n at \n6:00\n PM. All data in this report are preliminary and subject to change as cases continue to be investigated. \n These data include cases in NYC residents and foreign residents treated in NYC facilities.\n This table shows only confirmed deaths\n. A death is considered co\nnfirmed when the person \nhad a positive COVID\n-19 laboratory test.\n   NYC COVID\n-19 Deaths Among Confirmed Cases\n . Underlying \nConditions\n1  No \nUnderlying \nConditions \n Underlying \nConditions \nUnknown \n Total \n Age Group\n     -  0 to 17\n 6 0 0 6 -  18 to 44\n 447 15 95 557 -  45 to 64\n 2657 63 399 3119 -  65 to 74\n 2592 5 861 3458 -  75 and over\n 4829 2 1966 6797 -  Unknown\n 0 0 1 1 Sex\n     -  Female\n 4112 10 1324 5446 -  Male\n 6404 75 1992 8471 -  Unknown\n 15 0 6 21 Borough\n     -  Bronx\n 2725 15 306 3046 -  Brooklyn\n 3008 21 1165 4194 - 

## Extract the COVID Deaths and Parse

In [16]:
# Parse the deathspage text into a set of records
tblText = re.sub("Coronavirus.*0 to 17", "0 to 17", deathspage, flags=re.DOTALL)
tblText = re.sub("Total.*","", tblText, flags=re.DOTALL)
tblText = re.sub("\n", ",", tblText, flags=re.DOTALL)
tblText = re.sub("0 to 17", "0-17", tblText, flags=re.DOTALL)
tblText = re.sub("18 to 44", "18-44", tblText, flags=re.DOTALL)
tblText = re.sub("45 to 64", "45-64", tblText, flags=re.DOTALL)
tblText = re.sub("65 to 74", "65-74", tblText, flags=re.DOTALL)
tblText = re.sub("75 and over", "Over_75", tblText, flags=re.DOTALL)
tblText = re.sub(" -.+?(?=[a-zA-Z0-9])", ";", tblText, flags=re.DOTALL)
tblText = re.sub(", ", ",", tblText, flags=re.DOTALL)
tblText = re.sub(" ", ",", tblText, flags=re.DOTALL)
tblText = re.sub(",Sex.+?(?=;)", "", tblText, flags=re.DOTALL)
tblText = re.sub(",Borough.+?(?=;)", "", tblText, flags=re.DOTALL)
tblText = re.sub("Unknown.+?(?=[A-Z])", "", tblText, flags=re.DOTALL)
tblText = re.sub("Unknown.*", "", tblText, flags=re.DOTALL)
tblText = re.sub("Staten,Island", "Staten_Island", tblText, flags=re.DOTALL)


In [17]:
# Display parsed text to check format
tblText

'0-17,6,0,0,6;18-44,447,15,95,557;45-64,2657,63,399,3119;65-74,2592,5,861,3458;Over_75,4829,2,1966,6797;Female,4112,10,1324,5446;Male,6404,75,1992,8471;Bronx,2725,15,306,3046;Brooklyn,3008,21,1165,4194;Manhattan,1207,7,608,1822;Queens,3144,42,1027,4213;Staten_Island,441,0,214,655;'

In [18]:
# Split the extracted text into rows and remove any rows that do not contain data
rows = re.split(";", tblText)
rows = [row for row in rows if re.search(r"[0-9]",row)!=None]

In [19]:
rows

['0-17,6,0,0,6',
 '18-44,447,15,95,557',
 '45-64,2657,63,399,3119',
 '65-74,2592,5,861,3458',
 'Over_75,4829,2,1966,6797',
 'Female,4112,10,1324,5446',
 'Male,6404,75,1992,8471',
 'Bronx,2725,15,306,3046',
 'Brooklyn,3008,21,1165,4194',
 'Manhattan,1207,7,608,1822',
 'Queens,3144,42,1027,4213',
 'Staten_Island,441,0,214,655']

In [20]:
# Convert text into a list of rows
dftemp = []
for row in rows:
    record = row.split(",")
    del record[-1]
    for i in range(1,4):
        record[i] = int(record[i])
    if (record[0][0].isdigit() or record[0][0]=='O'):
        record.append('age')
    elif (record[0]=='Female' or record[0]=='Male'):
        record.append('sex')
    else:
        record.append('borough')
    dftemp.append(record)
   

In [21]:
# Transpose list of rows into higher normal form
deathsdata = []
deathtype = ['death_underlying','death_no_underlying','death_underlying_pending']
for row in dftemp:
    for i in range(1,4):
        record = []
        record.append(row[-1])
        record.append(row[0])
        record.append(deathtype[i-1])
        record.append(row[i])
        deathsdata.append(record)
   

In [22]:
# Convert list into pandas dataframe and display
deathsdf = pd.DataFrame(deathsdata, columns=['feature','category','type','count'])
deathsdf['date']=date(yr,mo,day)
deathsdf

,feature,category,type,count,date
0,age,0-17,death_underlying,6,2020-05-05
1,age,0-17,death_no_underlying,0,2020-05-05
2,age,0-17,death_underlying_pending,0,2020-05-05
3,age,18-44,death_underlying,447,2020-05-05
4,age,18-44,death_no_underlying,15,2020-05-05
5,age,18-44,death_underlying_pending,95,2020-05-05
6,age,45-64,death_underlying,2657,2020-05-05
7,age,45-64,death_no_underlying,63,2020-05-05
8,age,45-64,death_underlying_pending,399,2020-05-05
9,age,65-74,death_underlying,2592,2020-05-05


In [23]:
# Concatenate cases and deaths dataframes and display
dailydf = pd.concat([casesdf, deathsdf]).reset_index().drop(columns=['index'])
dailydf

,category,count,feature,type,date
0,0-17,4188,age,cases,2020-05-05
1,18-44,63726,age,cases,2020-05-05
2,45-64,63670,age,cases,2020-05-05
3,65-74,21590,age,cases,2020-05-05
4,Over_75,19779,age,cases,2020-05-05
5,Female,83891,sex,cases,2020-05-05
6,Male,89195,sex,cases,2020-05-05
7,Bronx,39587,borough,cases,2020-05-05
8,Brooklyn,46139,borough,cases,2020-05-05
9,Manhattan,21470,borough,cases,2020-05-05


In [28]:
# Save the dataframe to a CSV file
#yrstr = str(yr)
#mostr = str(mo) if mo >= 10 else ('0'+str(mo))
#daystr = str(day) if day >= 10 else ('0'+str(day))
#datestr = yrstr + mostr + daystr
#fname = datestr + '-NYCHealth-Daily-COVID-data.csv'

In [29]:
#dailydf.to_csv(fname,sep=",",index=False)

## Read in NYC Hospitalization Data, Extract and Parse

In [24]:
# Read in text from PDF and show in raw format
url = 'https://www1.nyc.gov/assets/doh/downloads/pdf/imm/covid-19-daily-data-summary-hospitalizations-05062020-1.pdf'
localurl = 'covid-19-daily-data-summary-hospitals-200330.pdf'
hosppage = textfrompdf(url,readfrom='online')
hosppage

'Coronavirus Disease 2019 (COVID\n-19) \n  Daily Data Summary\n  The data in this report reflect events and activities\n as of\n May \n5, 2020 at \n6:00\n PM. \n All data in this report are preliminary and subject to change as cases continue to be investigated. \n These\n data \ninclude \ncases in NYC residents and foreign residents treated in NYC facilities\n.  NYC COVID\n-19 Hospitalizations Among Confirmed Cases\n  . Ever \n Hospitalized Cases\n1 All Cases\n Age Group\n   -  0 to 17\n 303 (7%)\n 4188 -  18 to 44\n 6145 (10%)\n 63726 -  45 to 64\n 15007 (24%)\n 63670 -  65 to 74\n 9936 (46%)\n 21590 -  75 and over\n 12281 (62%)\n 19779 -  Unknown\n 4 (1%)\n 335 Sex\n   -  Female\n 18237 (22%)\n 83891 -  Male\n 25319 (28%)\n 89195 -  Unknown\n 120 (59%)\n 202 Borough\n   -  Bronx\n 10147 (26%)\n 39587 -  Brooklyn\n 11641 (25%)\n 46139 -  Manhattan\n 5917 (28%)\n 21470 -  Queens\n 14003 (26%)\n 53692 -  Staten Island\n 1948 (16%)\n 12311 -  Unknown\n 20 (22%)\n 89 Total\n 43676 (25%)\n

In [25]:
# Parse the deathspage text into a set of records
tblText = re.sub("Coronavirus.*0 to 17", "0 to 17", hosppage, flags=re.DOTALL)
tblText = re.sub("Total.*","", tblText, flags=re.DOTALL)
tblText = re.sub("\n ", ",", tblText, flags=re.DOTALL)
tblText = re.sub(" \([0-9]{1,3}%\),", ",", tblText, flags=re.DOTALL)
tblText = re.sub(" -.+?(?=[a-zA-Z0-9])", ";", tblText, flags=re.DOTALL)
tblText = re.sub(" Sex.+?(?=;)", "", tblText, flags=re.DOTALL)
tblText = re.sub(" Borough.+?(?=;)", "", tblText, flags=re.DOTALL)
tblText = re.sub("0 to 17", "0-17", tblText, flags=re.DOTALL)
tblText = re.sub("18 to 44", "18-44", tblText, flags=re.DOTALL)
tblText = re.sub("45 to 64", "45-64", tblText, flags=re.DOTALL)
tblText = re.sub("65 to 74", "65-74", tblText, flags=re.DOTALL)
tblText = re.sub("75 and over", "Over_75", tblText, flags=re.DOTALL)
tblText = re.sub(" \n", "_", tblText, flags=re.DOTALL)
tblText = re.sub("Unknown.+?(?=F)", "", tblText, flags=re.DOTALL)
tblText = re.sub("Unknown.+?(?=B)", "", tblText, flags=re.DOTALL)
tblText = re.sub("Unknown.*", "", tblText, flags=re.DOTALL)


In [26]:
tblText

'0-17,303,4188;18-44,6145,63726;45-64,15007,63670;65-74,9936,21590;Over_75,12281,19779;Female,18237,83891;Male,25319,89195;Bronx,10147,39587;Brooklyn,11641,46139;Manhattan,5917,21470;Queens,14003,53692;Staten Island,1948,12311;'

In [27]:
# Split the extracted text into rows and remove any rows that do not contain data
rows = re.split(";", tblText)
rows = [row for row in rows if re.search(r"[0-9]",row)!=None]
rows

['0-17,303,4188',
 '18-44,6145,63726',
 '45-64,15007,63670',
 '65-74,9936,21590',
 'Over_75,12281,19779',
 'Female,18237,83891',
 'Male,25319,89195',
 'Bronx,10147,39587',
 'Brooklyn,11641,46139',
 'Manhattan,5917,21470',
 'Queens,14003,53692',
 'Staten Island,1948,12311']

In [28]:
# Convert text into a list of rows
dftemp = []
for row in rows:
    record = row.split(",")
    del record[-1]
    for i in range(1,1):
        record[i] = int(record[i])
    if (record[0][0].isdigit() or record[0][0]=='O'):
        record.append('age')
    elif (record[0]=='Female' or record[0]=='Male'):
        record.append('sex')
    else:
        record.append('borough')
    dftemp.append(record)
   

In [29]:
dftemp

[['0-17', '303', 'age'],
 ['18-44', '6145', 'age'],
 ['45-64', '15007', 'age'],
 ['65-74', '9936', 'age'],
 ['Over_75', '12281', 'age'],
 ['Female', '18237', 'sex'],
 ['Male', '25319', 'sex'],
 ['Bronx', '10147', 'borough'],
 ['Brooklyn', '11641', 'borough'],
 ['Manhattan', '5917', 'borough'],
 ['Queens', '14003', 'borough'],
 ['Staten Island', '1948', 'borough']]

In [30]:
# Convert list into pandas dataframe and display
hospdf = pd.DataFrame(dftemp, columns=['category','count','feature'])
hospdf['date']=date(yr,mo,day)
hospdf['type']='hospitalizations'
hospdf

,category,count,feature,date,type
0,0-17,303,age,2020-05-05,hospitalizations
1,18-44,6145,age,2020-05-05,hospitalizations
2,45-64,15007,age,2020-05-05,hospitalizations
3,65-74,9936,age,2020-05-05,hospitalizations
4,Over_75,12281,age,2020-05-05,hospitalizations
5,Female,18237,sex,2020-05-05,hospitalizations
6,Male,25319,sex,2020-05-05,hospitalizations
7,Bronx,10147,borough,2020-05-05,hospitalizations
8,Brooklyn,11641,borough,2020-05-05,hospitalizations
9,Manhattan,5917,borough,2020-05-05,hospitalizations


In [31]:
# Concatenate cases and deaths dataframes and display
dailydf = pd.concat([casesdf, deathsdf, hospdf]).reset_index().drop(columns=['index'])
dailydf

,category,count,feature,type,date
0,0-17,4188,age,cases,2020-05-05
1,18-44,63726,age,cases,2020-05-05
2,45-64,63670,age,cases,2020-05-05
3,65-74,21590,age,cases,2020-05-05
4,Over_75,19779,age,cases,2020-05-05
5,Female,83891,sex,cases,2020-05-05
6,Male,89195,sex,cases,2020-05-05
7,Bronx,39587,borough,cases,2020-05-05
8,Brooklyn,46139,borough,cases,2020-05-05
9,Manhattan,21470,borough,cases,2020-05-05


In [32]:
# Save the dataframe to a CSV file
yrstr = str(yr)
mostr = str(mo) if mo >= 10 else ('0'+str(mo))
daystr = str(day) if day >= 10 else ('0'+str(day))
datestr = yrstr + mostr + daystr
fname = datestr + '-NYCHealth-Daily-COVID-data.csv'

In [33]:
dailydf.to_csv(fname,sep=",",index=False)